In [ ]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from kafka import KafkaProducer
import time

#comando para ejecutar en local
#kafka-topics --create --topic rk_hadoop --bootstrap-server localhost:9092 --partitions 1 --replication-factor 1

# Credenciales de autenticación de Twitter
access_token = ""
access_token_secret = ""
consumer_key = ""
consumer_secret = ""

# Configuración de Kafka local
kafka_endpoint = "localhost:9092"
kafka_topic = "rk_hadoop"
twitter_hash_tag = ["Java"]
time_limit = 10

class StdOutListener(StreamListener):
    def __init__(self, time_limit=time_limit):
        self.start_time = time.time()
        self.limit = time_limit
        super(StdOutListener, self).__init__()

    def on_data(self, data):
        if (time.time() - self.start_time) < self.limit:
            # Envío de mensajes a Kafka
            producer.send(kafka_topic, value=data.encode('utf-8'))
            print(data)
            return True
        else:
            print("Tiempo límite alcanzado.")
            return False

    def on_error(self, status):
        print("Error:", status)

# Inicializar el productor de Kafka
producer = KafkaProducer(bootstrap_servers=kafka_endpoint)

# Configuración de autenticación de Tweepy
l = StdOutListener()
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Inicializar el flujo de Tweepy
stream = Stream(auth, l)

# Filtrar el flujo por el hashtag especificado
stream.filter(track=twitter_hash_tag)